In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/drowsy-model-v1/drowsy_model.pth
/kaggle/input/yawn-eye-dataset-new/dataset_new/test/Closed/_163.jpg
/kaggle/input/yawn-eye-dataset-new/dataset_new/test/Closed/_330.jpg
/kaggle/input/yawn-eye-dataset-new/dataset_new/test/Closed/_498.jpg
/kaggle/input/yawn-eye-dataset-new/dataset_new/test/Closed/_167.jpg
/kaggle/input/yawn-eye-dataset-new/dataset_new/test/Closed/_76.jpg
/kaggle/input/yawn-eye-dataset-new/dataset_new/test/Closed/_439.jpg
/kaggle/input/yawn-eye-dataset-new/dataset_new/test/Closed/_226.jpg
/kaggle/input/yawn-eye-dataset-new/dataset_new/test/Closed/_168.jpg
/kaggle/input/yawn-eye-dataset-new/dataset_new/test/Closed/_96.jpg
/kaggle/input/yawn-eye-dataset-new/dataset_new/test/Closed/_64.jpg
/kaggle/input/yawn-eye-dataset-new/dataset_new/test/Closed/_373.jpg
/kaggle/input/yawn-eye-dataset-new/dataset_new/test/Closed/_115.jpg
/kaggle/input/yawn-eye-dataset-new/dataset_new/test/Closed/_20.jpg
/kaggle/input/yawn-eye-dataset-new/dataset_new/test/Closed/_559.jpg
/kagg

In [2]:
import torch.nn as nn
class DrowsinessDetector(nn.Module):
    def __init__(self):
        super(DrowsinessDetector, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64*56*56,128)
        self.fc2 = nn.Linear(128,2)
        self.dropout = nn.Dropout(p=0.5)
        
    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = self.pool(x)
        x = x.view(-1, 64*56*56)
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x
        

In [3]:
import torch

# Define the path to your saved model file
model_path = '/kaggle/input/drowsy-model-v1/drowsy_model.pth'

# Load the saved model
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

model = DrowsinessDetector()
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

DrowsinessDetector(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=200704, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [4]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

# Define the transforms to be applied to each image
test_transforms = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load the test dataset from the new directory structure
test_dataset = ImageFolder('/kaggle/input/yawn-eye-dataset-new/dataset_new/test', transform=test_transforms)

# Class labels for the new dataset
class_labels = ['Non Drowsy', 'Drowsy']

# Create a dictionary to map the new class labels to the original class labels
label_map = {
    0: {'Open', 'no_yawn'},
    1: {'Closed', 'yawn'}
}

# Create a new dataset with the updated class labels
new_test_dataset = []
for image, label in test_dataset:
    new_label = None
    for key, value in label_map.items():
        if test_dataset.classes[label] in value:
            new_label = key
            break
    new_test_dataset.append((image, new_label))

# Define a dataloader to load the new test dataset
batch_size = 32
test_dataloader = torch.utils.data.DataLoader(new_test_dataset, batch_size=batch_size, shuffle=False)


In [5]:
# Evaluate the model on the test dataset
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_dataloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print('Accuracy of the model on the test dataset: {:.2f}%'.format(accuracy))
    

Accuracy of the model on the test dataset: 49.42%


Re-Train the model

In [6]:


# Define the transforms to be applied to each image
dataset_transforms = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load the test dataset from the new directory structure
train_dataset = ImageFolder('/kaggle/input/yawn-eye-dataset-new/dataset_new/train', transform=test_transforms)

# Class labels for the new dataset
class_labels = ['Non Drowsy', 'Drowsy']

# Create a dictionary to map the new class labels to the original class labels
label_map = {
    0: {'Open', 'no_yawn'},
    1: {'Closed', 'yawn'}
}

# Create a new dataset with the updated class labels
new_train_dataset = []
for image, label in train_dataset:
    new_label = None
    for key, value in label_map.items():
        if train_dataset.classes[label] in value:
            new_label = key
            break
    new_train_dataset.append((image, new_label))

# Split the data into training and validation sets
torch.manual_seed(42)
train_size = int(0.8*len(new_train_dataset))
val_size = len(new_train_dataset)- train_size

train_dataset, val_dataset = torch.utils.data.random_split(
    new_train_dataset, [train_size, val_size])

    
# Define a dataloader to load the new train dataset
batch_size = 32
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = batch_size)

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import models
from torchvision.transforms import transforms
from torchvision.datasets import ImageFolder

# Initalize the model
model = DrowsinessDetector()

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [8]:
# Train the model on GPU

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# Move the model to the device
model = model.to(device)


num_epochs = 40
best_acc = 0.0

# Train phase
for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch + 1, num_epochs))
    print('-' * 10)

    model.train()  # set model to training mode

    running_loss = 0.0
    running_corrects = 0
    for inputs, labels in train_loader:
        inputs = inputs.to(device)  # move data to device
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = running_corrects.double() / len(train_loader.dataset)
    print('Train Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))

    # Evaluation phase
    model.eval()  # set model to evaluation mode
    running_loss = 0.0
    running_corrects = 0
    for inputs, labels in val_loader:
        inputs = inputs.to(device)  # move data to device
        labels = labels.to(device)

        with torch.no_grad():
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(val_loader.dataset)
    epoch_acc = running_corrects.double() / len(val_loader.dataset)
    print('Val Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))
    
    # Check if this is the best model so far
    if epoch_acc > best_acc:
        best_acc = epoch_acc
        best_model_wts = model.state_dict()

print('Training complete. Best validation accuracy: {:.4f}'.format(best_acc))
model.load_state_dict(best_model_wts)

Using device: cuda
Epoch 1/40
----------
Train Loss: 0.9013 Acc: 0.5773
Val Loss: 0.6069 Acc: 0.6680
Epoch 2/40
----------
Train Loss: 0.5329 Acc: 0.7202
Val Loss: 0.4833 Acc: 0.7368
Epoch 3/40
----------
Train Loss: 0.4408 Acc: 0.7775
Val Loss: 0.4095 Acc: 0.7874
Epoch 4/40
----------
Train Loss: 0.3781 Acc: 0.8099
Val Loss: 0.3827 Acc: 0.8117
Epoch 5/40
----------
Train Loss: 0.3401 Acc: 0.8459
Val Loss: 0.4137 Acc: 0.8036
Epoch 6/40
----------
Train Loss: 0.3141 Acc: 0.8520
Val Loss: 0.3555 Acc: 0.8360
Epoch 7/40
----------
Train Loss: 0.2745 Acc: 0.8789
Val Loss: 0.2798 Acc: 0.8806
Epoch 8/40
----------
Train Loss: 0.2130 Acc: 0.9103
Val Loss: 0.2426 Acc: 0.8846
Epoch 9/40
----------
Train Loss: 0.1679 Acc: 0.9311
Val Loss: 0.2473 Acc: 0.9028
Epoch 10/40
----------
Train Loss: 0.1366 Acc: 0.9463
Val Loss: 0.2240 Acc: 0.9089
Epoch 11/40
----------
Train Loss: 0.1135 Acc: 0.9539
Val Loss: 0.3594 Acc: 0.8765
Epoch 12/40
----------
Train Loss: 0.1228 Acc: 0.9508
Val Loss: 0.2256 Acc: 0

<All keys matched successfully>

In [9]:
# Evaluate the model on the test dataset
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_dataloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print('Accuracy of the model on the test dataset: {:.2f}%'.format(accuracy))

Accuracy of the model on the test dataset: 94.92%


In [10]:
torch.save(model.state_dict(), '/kaggle/working/drowsy-v2.pth')